*  count file for statistics


In [20]:
import pickle
import numpy as np
import pandas as pd
import os,sys
import random
# from tqdm import tqdm

In [31]:
root_dir = "../0112_0125/"
hp_date_first = False #先分日期才分四種hp?
pcap_date_dir = True #pcap資料夾有多一層日期資料夾?

In [32]:
isps = next(os.walk(root_dir))[1]
if 'csv' in isps:
    isps.remove('csv')
if 'meta_data' in isps:
    isps.remove('meta_data')
# pcap date
all_dates = []
for isp in isps:
#     ori_dirs = next(os.walk(root_dir + isp))[1] #honeypot/pcap
    pcap_dir = root_dir + isp + '/pcap/'
    hp_dir = root_dir + isp + '/honeypot/'
    if pcap_date_dir == False:
        pcap_files = next(os.walk(pcap_dir))[2]
        pcap_files = list(filter(lambda f: f.endswith(".tar.gz"), pcap_files))
        dates = [x.split('.')[1] for x in pcap_files]
        dates = [x.replace('-','_') for x in dates]
        all_dates.extend(dates)
    else:
        pcap_dates = next(os.walk(pcap_dir))[1] #csv/meta_data need to be excluded
        all_dates.extend(pcap_dates)
    if hp_date_first:
        hp_dates = next(os.walk(hp_dir))[1]
        dates = [x.replace('-','_') for x in hp_dates]
        all_dates.extend(dates)
    else:
        hp_types_dir = next(os.walk(hp_dir))[1]
        for hp_type in hp_types_dir:
            hp_dates = next(os.walk(hp_dir + hp_type + '/'))[1]
            all_dates.extend(hp_dates)
        
#     for o_d in ori_dirs:
all_dates = sorted(list(set(all_dates)))
all_dates

['2020_01_12',
 '2020_01_13',
 '2020_01_14',
 '2020_01_15',
 '2020_01_16',
 '2020_01_17',
 '2020_01_18',
 '2020_01_19',
 '2020_01_20',
 '2020_01_21',
 '2020_01_22',
 '2020_01_23',
 '2020_01_24',
 '2020_01_25']

In [33]:
pickle.dump(obj=all_dates,file=open(root_dir+'dates.pkl','wb'))

In [34]:
df = pd.DataFrame(columns=['ISP', 'pcap_loss_date',
                           'amun_loss_date','cowrie_loss_date','dionaea_loss_date','glastopf_loss_date'])
for isp in isps:
    pcap_dir = root_dir + isp + '/pcap/'
    hp_dir = root_dir + isp + '/honeypot/'
    pcap_loss = []
    amun_loss = []
    cowire_loss = []
    dionaea_loss = []
    glastopf_loss = []
    if pcap_date_dir == False:
        pcap_files = list(filter(lambda f: f.endswith(".tar.gz"), pcap_files))
        dates = [x.split('.')[1] for x in pcap_files]
        dates = [x.replace('-','_') for x in dates]
        for a_date in all_dates:
            if a_date not in dates:
                pcap_loss.append(a_date)
    else:
        pcap_dates = next(os.walk(pcap_dir))[1]
        for a_date in all_dates:
            if a_date not in pcap_dates:
                pcap_loss.append(a_date)
        for date in pcap_dates:
            if len(os.listdir(pcap_dir+date) ) == 0:
                pcap_loss.append(date)
    if hp_date_first:
        hp_dates = next(os.walk(hp_dir))[1]
        dates = [x.replace('-','_') for x in hp_dates]
        for a_date in all_dates:
            if a_date not in dates:
#                 print(isp)
                amun_loss.append(a_date)
                cowire_loss.append(a_date)                        
                dionaea_loss.append(a_date)                        
                glastopf_loss.append(a_date)
        for date in hp_dates:
            try:
                if len(os.listdir(hp_dir+date+'/amun') ) == 0:
                    amun_loss.append(date)
            except FileNotFoundError:
                amun_loss.append(date)
            try:
                if len(os.listdir(hp_dir+date+'/cowrie') ) == 0:
                    cowire_loss.append(date)
            except FileNotFoundError:
                cowire_loss.append(date)
            try:
                if len(os.listdir(hp_dir+date+'/dionaea') ) == 0:
                    dionaea_loss.append(date)
            except FileNotFoundError:
                dionaea_loss.append(date)
            try:
                if len(os.listdir(hp_dir+date+'/glastopf') ) == 0:
                    glastopf_loss.append(date)
            except FileNotFoundError:
                glastopf_loss.append(date)
    else:
        for a_date in all_dates:
            try:
                if len(os.listdir(hp_dir+'amun/'+a_date) ) == 0:
                    amun_loss.append(a_date)
            except FileNotFoundError:
                amun_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'cowrie/'+a_date) ) == 0:
                    cowire_loss.append(a_date)
            except FileNotFoundError:
                cowire_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'dionaea/'+a_date) ) == 0:
                    dionaea_loss.append(a_date)
            except FileNotFoundError:
                dionaea_loss.append(a_date)
            try:
                if len(os.listdir(hp_dir+'glastopf/'+a_date) ) == 0:
                    glastopf_loss.append(a_date)
            except FileNotFoundError:
                glastopf_loss.append(a_date)
    temp = [isp, "、".join(pcap_loss), 
            "、".join(amun_loss),"、".join(cowire_loss),"、".join(dionaea_loss),"、".join(glastopf_loss)]
    temp = pd.Series(temp,index=df.columns)
    df = df.append(temp, ignore_index=True)
                
df


,ISP,pcap_loss_date,amun_loss_date,cowrie_loss_date,dionaea_loss_date,glastopf_loss_date
0,中華電信,,,,,
1,中嘉寬頻,,,,,
2,台固媒體,,,2020_01_17,,
3,台灣大哥大,,,,,
4,台灣之星,,,,,
5,台灣固網,,,,,
6,台灣基礎開發,,,2020_01_12、2020_01_13、2020_01_16、2020_01_22、20...,,
7,台灣碩網,,2020_01_13、2020_01_25,2020_01_25,2020_01_25,2020_01_25
8,亞太電信,,2020_01_25,2020_01_25,2020_01_25,2020_01_25
9,凱擘,,,,,


In [35]:
pickle.dump(obj=df,file=open(root_dir+'lost_dates.pkl','wb'))